In [1]:
# Cell 1: Import các thư viện cần thiết
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import math
import warnings
warnings.filterwarnings('ignore')


KeyboardInterrupt: 

In [ ]:
# Cell 2: Đọc file Apple_clean.csv và xem cấu trúc dữ liệu
df = pd.read_csv('Apple_clean.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)

print(df.head())
print("\nCác cột trong dataset:", df.columns.tolist())


In [ ]:
# Cell 3: Chọn cột 'Close' và chuẩn hóa với MinMaxScaler
data = df[['Close']]
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data.values.reshape(-1, 1))

print("Dữ liệu sau khi scale:\n", scaled_data[:5])


In [ ]:
# Cell 4: Chia dữ liệu theo tỷ lệ 60/20/20
train_size = int(len(scaled_data) * 0.6)
val_size = int(len(scaled_data) * 0.2)
test_size = len(scaled_data) - train_size - val_size

train_data = scaled_data[:train_size]
val_data = scaled_data[train_size:train_size + val_size]
test_data = scaled_data[train_size + val_size:]

print(f"Train: {len(train_data)}, Validate: {len(val_data)}, Test: {len(test_data)}")


In [ ]:
# Cell 5: Hàm tạo tập dữ liệu có dạng (X, y) cho RNN
def create_dataset(dataset, time_step=60):
    X, y = [], []
    for i in range(len(dataset) - time_step - 1):
        X.append(dataset[i:(i + time_step), 0])
        y.append(dataset[i + time_step, 0])
    return np.array(X), np.array(y)


In [ ]:
# Cell 6: Tạo dữ liệu train/val/test có định dạng phù hợp với RNN
time_step = 60
X_train, y_train = create_dataset(train_data, time_step)
X_val, y_val = create_dataset(val_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

# Reshape về dạng 3D [samples, time_steps, features]
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

print("X_train shape:", X_train.shape)


In [ ]:
# Cell 7: Xây dựng mô hình LSTM RNN
model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(time_step, 1)),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()


In [ ]:
# Cell 8: Huấn luyện mô hình trên tập train, dùng early stopping
es = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
    verbose=1,
    callbacks=[es]
)


In [ ]:
# Cell 9: Vẽ biểu đồ training/validation loss
plt.figure(figsize=(10,5))
plt.plot(history.history['loss'], label='Train Loss', linewidth=2)
plt.plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
plt.title('RNN Model Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Cell 10: Dự đoán trên tập test
y_pred = model.predict(X_test)
y_pred_inv = scaler.inverse_transform(y_pred)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1,1))


In [ ]:
# Cell 11: Tính các chỉ số MAE, MSE, RMSE, MAPE
mae = mean_absolute_error(y_test_inv, y_pred_inv)
mse = mean_squared_error(y_test_inv, y_pred_inv)
rmse = math.sqrt(mse)
mape = mean_absolute_percentage_error(y_test_inv, y_pred_inv) * 100

print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"RMSE: {rmse:.4f}")
print(f"MAPE: {mape:.2f}%")


In [ ]:
# Cell 12: Vẽ biểu đồ so sánh giá trị thực tế và dự đoán
plt.figure(figsize=(14,6))
plt.plot(y_test_inv, label='Actual Close', linewidth=2)
plt.plot(y_pred_inv, label='Predicted Close', linewidth=2)
plt.title('RNN Model Prediction vs Actual (Apple)')
plt.xlabel('Time')
plt.ylabel('Close Price')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
# Cell 13: Dự đoán 30 ngày tiếp theo (fix chiều dữ liệu)
last_60_days = scaled_data[-60:]
pred_input = last_60_days.reshape(1, -1, 1)  # shape (1,60,1)
future_predictions = []

for _ in range(30):
    pred = model.predict(pred_input)            # pred shape (1,1)
    future_predictions.append(pred[0, 0])       # lưu giá trị scalar

    # append vào pred_input: bỏ first timestep, thêm pred dưới dạng (1,1,1)
    new_step = np.array([[[pred[0, 0]]]])       # shape (1,1,1)
    pred_input = np.append(pred_input[:, 1:, :], new_step, axis=1)

future_predictions = np.array(future_predictions).reshape(-1, 1)
future_predictions_inv = scaler.inverse_transform(future_predictions)


In [ ]:
# Cell 14: Tạo index thời gian cho 30 ngày tiếp theo
last_date = df.index[-1]
future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=30)

future_df = pd.DataFrame({
    'Date': future_dates,
    'Predicted_Close': future_predictions_inv.flatten()
}).set_index('Date')

future_df.head()


In [ ]:
# Cell 15: Vẽ toàn bộ biểu đồ (Train, Test, Predict, Future)
train_plot = scaler.inverse_transform(train_data)
val_plot = scaler.inverse_transform(val_data)
test_plot = y_test_inv

plt.figure(figsize=(14,6))
plt.plot(df.index[:len(train_plot)], train_plot, label='Train Data', color='green')
plt.plot(df.index[len(train_plot):len(train_plot)+len(val_plot)], val_plot, label='Validate Data', color='orange')
plt.plot(df.index[-len(test_plot):], test_plot, label='Actual Test', color='blue')
plt.plot(df.index[-len(test_plot):], y_pred_inv, label='Predicted Test', color='red')
plt.plot(future_df.index, future_df['Predicted_Close'], label='Next 30 Days', color='purple', linestyle='--')
plt.title('RNN Prediction of Apple Closing Prices + Next 30 Days')
plt.xlabel('Date')
plt.ylabel('Close Price')
plt.legend()
plt.grid(True)
plt.show()
